In [ ]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import re
import copy
from tqdm import tqdm

In [ ]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware

# # keep only values near the end of 2021
# hardware_instances = hardware_instances[hardware_instances["date"] > "2020-07-01"]
# hardware_instances = hardware_instances[hardware_instances["date"] < "2020-11-01"]

# unique_values = hardware_instances["hardware"].value_counts()
# pairs = []
# for i in range(0, len(unique_values)):
#     # print(unique_values.index[i], unique_values[i])
#     pairs.append((unique_values.index[i], unique_values[i]))

# pairs = sorted(pairs, key=lambda x: x[0], reverse=False)
# for pair in pairs:
#     # print(pair)
#     print(pair[0])

pd.set_option('display.max_rows', None)

# keep only values near the end of 2021
hardware_instances = hardware_instances[hardware_instances["date"] > "2020-07-01"]
hardware_instances = hardware_instances[hardware_instances["date"] < "2020-12-01"]
hardware_instances[["date", "hardware"]].to_csv("hardware_instances_2020_all.csv", index=False)

from hardware_mapping import map_hardware_to_table
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))
hardware_instances

In [ ]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances[hardware_instances["date"] > "2018-01-01"]
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(map_hardware_to_table))

# find most common unique values that were mapped to "unknown"
unique_values = hardware_instances[hardware_instances["hardware_mapped"] == "unknown"]["hardware"].value_counts()
unique_values[:100]

In [ ]:
hardware_instances = pd.read_csv('hardware_instances.csv') # columns: date,hardware
hardware_instances = hardware_instances.assign(hardware_mapped = hardware_instances["hardware"].apply(lambda x: str(x).strip()).apply(map_hardware_to_table))

# print things that are "not found"
unique_values = hardware_instances[hardware_instances["hardware_mapped"] == "not found"]["hardware"].value_counts()
unique_values[:10]

In [8]:
instances = pd.read_csv('hardware_instances_inc_threads.csv') # columns: date,thread

# def print_all(instances, date1, date2):
#     filtered = instances[instances["date"] > date1]
#     filtered = filtered[filtered["date"] < date2]
#     for index, row in filtered.iterrows():
#         print(row["date"], row["thread"], "\n\n\n\n\n\n\n\n")
# print_all(instances, "2022-01-01", "2022-06-01")

efficiency = pd.read_csv('hardware_instances_with_efficiency.csv') # columns: date,hardware_name,TH/J
efficiency["ln_efficiency"] = np.log(efficiency["TH/J"])
efficiency = efficiency[efficiency["date"] > "2017-10-01"]
efficiency = efficiency[efficiency["ln_efficiency"] < -9]

for index, row in efficiency.iterrows():
    print(row["date"], row["hardware_name"], row["TH/J"], row["ln_efficiency"])
    filtered = instances[instances["date"] == row["date"]]
    print(filtered.iloc[0]["thread"])
    print("\n\n\n\n\n\n\n\n\n\n\n\n")

2017-11-27 16:34:40 x6500 fpga miner 2.325e-05 -10.669205425930908
User:
In the given Bitcoin forum thread, pay close attention to the language used when mentioning hardware pieces. Look for explicit statements indicating ownership or hypothetical discussions.

```thread
Date: 2017-11
Topic: X6500 Custom FPGA Miner
### Original post:
I am going to get mine in a couple hours! Will you have the software posted on the first post so we will be ready to go? Or does it come with a disk or something?What will need to be done to get it running?

### Reply 1:
Well, there's a software section on the website that appears to have software downloads and instructions, though I've no idea if that's the latest version and obviously don't have the hardware to actually use it.

### Reply 2:
Thanks for that! I will give it a shot later today and let everyone know!

### Reply 3:
What's the status on the Mac miner?

### Reply 4:
Thanks, makomk, for looking over my shoulder there! I meant to post a link to 